In [1]:
FOLDER = '../data/stl'
EXTENSION = '.stl'
OUTPUT_FOLDER = 'output/extra-measurements'

In [2]:
import sys
sys.path.append('..')

In [3]:
import os

files = os.listdir(FOLDER)
files = [os.path.join(FOLDER, f) for f in files if EXTENSION in f]
files.sort()
files

['../data/stl/020_l.stl',
 '../data/stl/15032021_001_l 1.stl',
 '../data/stl/15032021_001_r 1.stl']

## Example run

In [4]:
import pandas as pd

file = files[0]

# load labelling
lmrk = pd.concat([
    pd.read_pickle(file.replace(EXTENSION, '.pkl')),
    pd.read_pickle(file.replace(EXTENSION, '-extra-udmc.pkl')),
])
lmrk

coord,x,y,z
landmark,,,
P1,69.653463,54.673492,-7.735388
P10,-168.788818,79.678271,-11.032859
P11,-122.558701,102.675117,-60.435042
P12,-123.716329,31.840291,-67.462064
P2,59.080175,94.482407,-9.113344
P3,18.191420,5.494654,-10.807760
P4,19.126142,99.146723,-18.009934
P5,-15.352643,2.470859,-11.472470
P6,-34.886927,64.674735,-62.592272


In [5]:
import pyvista as pv
from mesh4d.analyse import crave
from measure import frame, visual

# local frame
mesh = crave.fix_pvmesh_disconnect(pv.read(file), lmrk.values)
axes_frame, origin = frame.estimate_foot_frame(mesh, lmrk)
mesh_clip = frame.foot_clip(mesh, lmrk)
mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
lmrk_local = frame.df2local(lmrk, axes_frame, origin)

# visual check
visual.plot_axes(origin, axes_frame, mesh_clip)

Widget(value="<iframe src='http://localhost:61004/index.html?ui=P_0x7fdbb238af70_0&reconnect=auto' style='widt…

In [6]:
lmrk_local

coord,x,y,z
landmark,,,
P1,95.204932,-0.753839,6.982687
P10,-144.178050,-12.993784,13.050465
P11,-98.644717,-39.191606,61.496780
P12,-95.904574,31.487068,69.578191
P2,82.524019,-39.955482,7.914767
P3,46.503865,51.058825,11.454183
P4,42.489928,-42.608438,17.266966
P5,13.181717,55.865117,12.604671
P6,-9.036342,-5.981267,63.057902


In [7]:
from measure.metric import height, dist, dist_along_axis

def al(
        lmkr_local: pd.DataFrame,
    ):
    """arch length"""
    return dist(lmkr_local, 'MLA-AP', 'MLA-PP')

def ab(
        lmkr_local: pd.DataFrame,
    ):
    """arch breadth"""
    return dist(lmkr_local, 'LMB', 'MMB')

def ad(
        lmkr_local: pd.DataFrame,
    ):
    """arch depth"""
    return dist_along_axis(lmkr_local, 'P4', 'MMB', 'x')

def ah(
        lmkr_local: pd.DataFrame,
    ):
    """arch height"""
    return height(lmkr_local, 'AA')

In [8]:
# metrics
results = {
    'file': ['description', str(file)],
    'AL': ['arch length (mm)', al(lmrk_local)],
    'AB': ['arch breadth (mm)', ab(lmrk_local)],
    'AD': ['arch depth (mm)', ad(lmrk_local)],
    'AH': ['arch height (mm)', ah(lmrk_local)],
}

df_extra = pd.DataFrame(results)
df_extra

,file,AL,AB,AD,AH
0,description,arch length (mm),arch breadth (mm),arch depth (mm),arch height (mm)
1,../data/stl/020_l.stl,103.602601,43.156748,66.702218,14.768467


### Visualization

In [9]:
settings = {
    'AL': ['MLA-AP', 'MLA-PP'],
    'AB': ['LMB', 'MMB'],
}

for name, [landmark1, landmark2] in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_dist(scene, lmrk_local, landmark1, landmark2, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value="<iframe src='http://localhost:61004/index.html?ui=P_0x7fdbb257a490_1&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:61004/index.html?ui=P_0x7fdbb7dac4f0_2&reconnect=auto' style='widt…

In [10]:
settings = {
    'AD': ['P4', 'MMB', 'x'],
}

for name, [landmark1, landmark2, axis] in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_dist_along_axis(scene, lmrk_local, landmark1, landmark2, axis, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value="<iframe src='http://localhost:61004/index.html?ui=P_0x7fdbb7d64dc0_3&reconnect=auto' style='widt…

In [11]:
settings = {
    'AH': 'AA',
}

for name, landmark in settings.items():
    scene = pv.Plotter()
    scene.add_mesh(mesh_local, opacity=0.1)
    visual.plot_height(scene, lmrk_local, landmark, name=name)
    scene.show()
    scene.screenshot(f'{OUTPUT_FOLDER}/{name}.png')

Widget(value="<iframe src='http://localhost:61004/index.html?ui=P_0x7fdbb7d64eb0_4&reconnect=auto' style='widt…

## Run all

In [12]:
os.path.join(
    OUTPUT_FOLDER,
    os.path.basename(file).replace(EXTENSION, '-extra.csv'),
    )

'output/extra-measurements/020_l-extra.csv'

In [13]:
from tqdm import tqdm
from measure import metric

for file in tqdm(files):
    # load labelling
    lmrk = pd.concat([
        pd.read_pickle(file.replace(EXTENSION, '.pkl')),
        pd.read_pickle(file.replace(EXTENSION, '-extra-udmc.pkl')),
    ])

    # local frame
    mesh = crave.fix_pvmesh_disconnect(pv.read(file), lmrk.values)
    axes_frame, origin = frame.estimate_foot_frame(mesh, lmrk)
    mesh_clip = frame.foot_clip(mesh, lmrk)
    mesh_local = frame.foot2local(mesh_clip, axes_frame, origin)
    lmrk_local = frame.df2local(lmrk, axes_frame, origin)

    # metrics
    results = {
        'file': ['description', str(file)],
        'AL': ['arch length (mm)', al(lmrk_local)],
        'AB': ['arch breadth (mm)', ab(lmrk_local)],
        'AD': ['arch depth (mm)', ad(lmrk_local)],
        'AH': ['arch height (mm)', ah(lmrk_local)],
    }

    df_extra = pd.DataFrame(results)
    df_extra = pd.DataFrame(results).set_index('file')

    output_path = os.path.join(
        OUTPUT_FOLDER,
        os.path.basename(file).replace(EXTENSION, '-extra.csv'),
        )
    df_extra.to_csv(output_path)

    # combine all results so far
    output_path = os.path.join(OUTPUT_FOLDER, 'measurements-extra.csv')
    df_all = metric.combine_measurement_csv(OUTPUT_FOLDER)
    df_all.to_csv(output_path)

100%|██████████| 3/3 [00:04<00:00,  1.47s/it]
